In [ ]:
import matplotlib.pyplot as plt
from types import SimpleNamespace

from autocrop.autocrop import *

In [ ]:
plt.rcParams["xtick.bottom"] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams["ytick.left"] = plt.rcParams['ytick.labelleft'] = False

In [ ]:
def find_plates(rgb_img, sorted_contours, n=3, area_ratio_h=0.30, area_ratio_l=0.15, **kwargs):
    """
    Find the largest n contours that have correct size. Failed to find any will terminate the program.

    Args:
        rgb_img (np.ndarray)    : A numpy array of the image.
        sorted_contours (list)  : A 3D numpy array of the selected background pixels or a region on a rgb image.
        n (int)                 : The number of plate expected to be found.
        area_ratio_h (float)    : The higest ratio of an area considered to be a plate.
        area_ratio_l (float)    : The lowest ratio of an area considered to be a plate.
        (output_dir) (str)      : (Optional) The directory of the output path.
        (file) (pathlib.Path)   : (Optional) The file basename of the output.

    Returns:
        None
    """
    plate_number = 1
    logging.debug(f"Finding for plate that the ratio falls between {area_ratio_l}-{area_ratio_h} ...")
    for i in range(min(len(sorted_contours), n)):
        contour = sorted_contours[i]
        x, y, w, h = cv.boundingRect(contour)
        area_ratio = (w * h) / (rgb_img.shape[0] * rgb_img.shape[1])
        logging.debug(f"Found plate with ratio {area_ratio}")
        if area_ratio >= area_ratio_l and area_ratio <= area_ratio_h:
            crop = rgb_img[y : y + h, x : x + w]
            if all(i in kwargs for i in ("output_dir", "file")):
                output_dir = kwargs.get("output_dir")
                file = kwargs.get("file")
                output = f"{output_dir}/{file.stem}_{plate_number}{file.suffix}"
                cv.imwrite(output, crop)
                logging.info(f"Write plate{plate_number} image to {output}")
            else:
                fig, ax = plt.subplots(figsize=(8, 8))
                plt.imshow(crop)
            plate_number += 1

    if plate_number == 1:
        logging.warning(f"Cannot found any contour which area falls between {area_ratio_l}-{area_ratio_h}.")
        sys.exit(2)

In [ ]:
logging.basicConfig(format="%(asctime)s autocrop %(levelname)s [%(filename)s] %(message)s", level="DEBUG")
logger = logging.getLogger()

In [ ]:
parent_dir = "original_images"

file = Path(f"{parent_dir}/3_1_30.jpg")             # Obvious colony
# file = Path(f"{parent_dir}/17_3_4.jpg")             # Obvious but some of the colony are not round
# file = Path(f"{parent_dir}/7_2_low_N.jpg")          # Obvious colony with cut agar
# file = Path(f"{parent_dir}/10_1_30C.jpg")           # Overgrowth
# file = Path(f"{parent_dir}/7_1_4C.jpg")             # Weak colony

In [ ]:
f = ContextFilter()
f.add_filename(file.name)
logger.addFilter(f)

## Find and clip the edge of the container

In [ ]:
# Load image and convert it to grayscale.
imageRGB = cv.imread(str(file), cv.COLOR_BGR2RGB)

# Scale the images to a specific resolution and fix the distortion.
imageRGB = scale_image(imageRGB, adj_width=4000, distortion_offset=0.95)

gray = cv.cvtColor(imageRGB, cv.COLOR_RGB2GRAY)
hsv = cv.cvtColor(imageRGB, cv.COLOR_RGB2HSV)

In [ ]:
# Downscales the grayscale image if it is too large to speed up largest inner rectangle detection.
if gray.shape[1] > 1000:
    gray, scaling_factor = scale_image(gray, adj_width=1000, get_scal_f=True)
    hsv = cv.resize(hsv, (gray.shape[1], gray.shape[0]), interpolation=cv.INTER_AREA)

In [ ]:
# Fill the container to improve the largest inner rectangle detection.
filled_gray = fill_container(gray)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(hsv)

In [ ]:
# Find the largest inner rectangle, which is the square dish.
try:
    logging.debug("Detecting container...")
    x, y, w, h = detect_largest_inner_rectangle(filled_gray)
    logging.debug("Got image that contains container only.")
except:
    logging.error("Timeout. Cannot found largest interior rectangle.")
    sys.exit(1)

In [ ]:
# Get lower region that contains markers. The values were obtained empirically.
lower_bound = np.array([0, 74, 190])
upper_bound = np.array([150, 210, 255])

# Detect markers. Only search on part of the image below the container.
hsv = hsv[y + h :, x : x + w]
mx, my, mw, mh = get_markers(hsv, lower_bound, upper_bound)

In [ ]:
if "scaling_factor" in locals():
    logging.debug("Upscale the coordinates and dimensions back to apply on the original image.")
    x, y, w, h = [int(i / scaling_factor) for i in [x, y, w, h]]
    mx, my, mw, mh = [int(i / scaling_factor) for i in [mx, my, mw, mh]]
    del scaling_factor

In [ ]:
# Get the dish container and markers.
image_container = imageRGB[y : y + h, x : x + w]
markers_region = imageRGB[y + h :, x : x + w]
marker = markers_region[my : my + mh, mx : mx + mw]

In [ ]:
# Shrink the image by 4% to remove the container's edges.
image_container = shrink_image(image_container, perc=4)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(image_container)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(marker)

## Find the 96-well plate

In [ ]:
# Increase contrast.
blur = cv.GaussianBlur(image_container, (5, 5), 0)
enhanced_img = increase_contrast(blur)
gray = cv.cvtColor(enhanced_img, cv.COLOR_RGB2GRAY)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(gray)

In [ ]:
# Color balance by the background and output to file.
_, th = cv.threshold(gray, 100, 255, cv.THRESH_BINARY)
signal_ratio = (th > 0).sum() / th.size

background_reference = get_background(th, image_container)
imageRGB = color_balance_RGBscaling(imageRGB, background_reference)

In [ ]:
signal_ratio

In [ ]:
param_dict = {"H": {"signal_ratio_target": 0.20, "signal_ratio_diff": 0.05},
              "M": {"signal_ratio_target": 0.11, "signal_ratio_diff": 0.04},
              "L": {"signal_ratio_target": 0.04, "signal_ratio_diff": 0.03}}

In [ ]:
args = SimpleNamespace()
args.param = None

In [ ]:
if args.param:
    select = args.param
else:
    if signal_ratio < param_dict["M"]["signal_ratio_target"] - param_dict["M"]["signal_ratio_diff"]:
        select = "L"
    elif signal_ratio > param_dict["M"]["signal_ratio_target"] + param_dict["M"]["signal_ratio_diff"]:
        select = "H"
    else:
        select = "M"

logging.info(f'Use "{select}" parameter set')

In [ ]:
# Find the proper threshold.
th = find_proper_threshold(gray, **param_dict[select])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(th)

In [ ]:
# Use opening to denoise.
kernel = np.ones((5, 5), np.uint8)
opening = cv.morphologyEx(th, cv.MORPH_OPEN, kernel, iterations=5)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(opening)

In [ ]:
# Find the proper iterations.
sorted_contours, image = find_proper_iterations_for_dilations(opening, iterations=5, get_img=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(image)

## Find and crop the rectangular contours

In [ ]:
args.n_plates = 3

In [ ]:
# Find, crop and output the first n largest contours.
image_container = color_balance_RGBscaling(image_container, background_reference)
find_plates(image_container, sorted_contours, n=args.n_plates)